In [17]:
import cobra
import pandas as pd
from cobra.flux_analysis import pfba

In [18]:
# Load a model from an SBML file
model=cobra.io.read_sbml_model('../model/iQY1018.xml')
df = pd.read_excel('../data/biolog_iQY1018.xlsx',index_col='Substance')

In [20]:
# substrates utilization
model.reactions.get_by_id('EXpd00027').bounds = (0,1000)
growth_rate_e0 = []

for met in list(df['model_id']):
        met = met.strip()
        with model:
            subm = cobra.Metabolite(met+'_e0')
            model.add_metabolites([subm])
            sink_rxn = model.add_boundary(subm, type='sink',lb=-10)
            model.objective = 'biomass0'
            m = model.metabolites.get_by_id(met+'_e0')
            try:
                fluxes = pfba(model).fluxes
            except:
                growth_rate_e0.append(0)
            else:
                growth_rate_e0.append(fluxes['biomass0'])

In [21]:
df['iQY1018_rate'] = growth_rate_e0
true_met = []
for mi in df.index:
    if df.loc[mi,'BIOLOG result'] == "+" and df.loc[mi,'iQY1018_rate'] > 0:
        true_met.append(mi)
    if df.loc[mi,'BIOLOG result'] == "-" and df.loc[mi,'iQY1018_rate'] == 0:
        true_met.append(mi)

print('accuracy of substrates utilization:',len(true_met))

accuracy of substrates utilization: 64
